In [1]:
'''
Date: 2022.03.24
Title: 
By: Kang Jin Seong
'''
''' ## 트리의 앙상블'''

'''### 랜덤 포레스트 '''
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [5]:
# 결정 트리의 큰 잠정 중 하나인 특성 중요도를 계산합니다.
# 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것입니다.
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 랜덤 포레스트는 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련한다.
# 하나의 특성에 과도하게 집중하지 않고 좀더 많은 특성이 훈련에 기여할 기회가 생긴다.
# 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 된다.

[0.23167441 0.50039841 0.26792718]


In [6]:
#랜덤 포레스트는 훈련세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정트리를 훈련함.
# 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있다.
# 이런 샘플을 OOB(out of bag)샘플 이라고 한다.
# 남은 샘플을 사용하여 결정트리를 평가 할 수있다. -> 검즘세트의 역활을 수행한다.

rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [7]:
'''###엑스트라 트리 '''
# 랜덤 포레스트: 부트스트랩 샘플을 사용, 엑스트라 트리: 전체 훈련세트를 사용 대신 노드를 분할 할때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.
# 하나의 결정트리에서 특성을 무작위로 분할 한다면 성능이 낮아 지지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증세트의 점수를 높이는 효과가 있다.
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [8]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [12]:
'''###그레이디언트 부스팅 '''
# 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방법
# 기본적으로 깊이가 3인 결정 트리를 100개 사용한다.
# 깊이가 얕은 결정트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대 할 수있다.
# 경사 하강법을 사용하여 트리를 앙상블에 추가한다.
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [13]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2,random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 결정트리 개수를 500개로 늘렸지만 과대적합을 잘 억제하고 있다.
# 학습률 learning_rate의 기본 값은 0.
# 매개 변수 중 하나인 트리 훈련에서 사용할 훈련 세트의 비율을 정하는 subsample
# 기본 값은 1.0으로 전체 훈련 세트를 사용한다. 하지만 1보다 작으면 일부를 사용하는데 이는 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는
# 확률적 경사 하강법 이나 미니배치 경사 하강법과 비슷하다.

0.9464595437171814 0.8780082549788999


In [14]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
# 순서대로 트리를 추가하기 때문에 훈련속도가 느리다.
# n_jobs 매개 변수가 없다.

[0.15872278 0.68010884 0.16116839]


In [16]:
'''### 히스트그램 기반 그레이디언트 부스팅'''
# 입력 특성을 256개의 구간으로 나눕니다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있습니다.
# 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용합니다. 따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리 할 필요가 없다.
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [18]:
from sklearn.inspection import permutation_importance # 특성 중요도 계산
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)  # 특성 중요도, 평균, 표준편차

[0.08876275 0.23438522 0.08027708]


In [19]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)
hgb.score(test_input, test_target)

[0.05969231 0.20238462 0.049     ]


0.8723076923076923